In [2]:
# import utils
import numpy as np
import pickle
from tqdm import tqdm
import os, sys
from sys import stdout

import keras
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model, load_model
from keras.layers import Input, Masking, TimeDistributed, Dense, Concatenate, Dropout, LSTM, GRU
from keras.optimizers import Adam

Using TensorFlow backend.
/home/vmarina/.conda/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vmarina/.conda/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vmarina/.conda/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vmarina/.conda/envs/py37/l

In [3]:
class DeepChopin:

    def __init__(self, corpus, TBPTT_size=128, batch_size=32):
        self.datapath = "./data/"
        self.corpus = corpus

        self.loadmodelpath = "./load/"
        if not os.path.exists("./save/"):
            os.makedirs("./save/")
        self.outpath = "./save/"+corpus+'/'
        if not os.path.exists(self.outpath):
            os.makedirs(self.outpath)
        self.genpath = self.outpath + "midi/"
        if not os.path.exists(self.genpath):
            os.makedirs(self.genpath)
        

        self.TBPTT_size = TBPTT_size
        self.batch_size = batch_size


        self.IO = None
        self.dataset = None
        
        self.log = {'loss': [], 'val_loss': [], 
                        'dT_acc': [], 'val_dT_acc': [], 
                        'T_acc': [], 'val_T_acc': [], 
                        'P_acc': [], 'val_P_acc': [], 
                        'dT_loss': [], 'val_dT_loss': [],
                        'T_loss': [], 'val_T_loss': [],
                        'P_loss': [], 'val_P_loss': []}

        self.best_val_accP = 0.
        self.best_tr_accP = 0.
        self.best_val_accT = 0.
        self.best_tr_accT = 0.
        self.best_val_accdT = 0.
        self.best_tr_accdT = 0.

        self.split = None
        self.maxlen = None


    def processData(self):
        dataset = {}
        print("Reading MIDI files")
        for filename in tqdm(os.listdir(self.datapath+self.corpus+"/midi/")):
            if filename[-3:] in ["mid", "MID", "SQU", "KPP", "squ", "kpp"]:
                label = filename[:-4]
                try:
                    dTseq, Tseq, Pseq, tpb = parseMIDI(self.datapath+self.corpus+"/midi/"+filename)

                except TypeError:
                    print(label, 'skipped')
                    continue
                dataset[label] = {}
                dataset[label]['T']= Tseq
                dataset[label]['dT']= dTseq
                dataset[label]['P']= Pseq
                dataset[label]['TPB'] = tpb
                dPseq = []
                for n, p in enumerate(Pseq):
                    if n == 0:
                        dPseq.append(0)
                    else:
                        dPseq.append(p-Pseq[n-1])
                dataset[label]['dP'] = dPseq

        #Construct all possible durations
        bases = [1./16, 1./12]#64 and triplet of 32
        increments = []
        for b in bases:
            m = 0
            while m*b < 1.:
                increments.append(m*b)
                m+=1
        allowed_durations = []
        for d in range(64): #For all durations up to 64 quarter notes
            for i in increments:
                allowed_durations.append(round(d+i, 5))
        increments = sorted(list(set(increments)))
        allowed_durations = sorted(list(set(allowed_durations)))

        print("Rhythm Normalization")
        #Convert to the set of possible durations
        for label, score in tqdm(dataset.items()):
            score['dT'] = [findClosest(dt, allowed_durations) for dt in score['dT']]
            score['T'] = [findClosest(t, allowed_durations) for t in score['T']]
            

        dictionaries = {}
        for key in ['dT', 'T', 'P']:
            flatten = []
            for label, score in dataset.items():
                for x in score[key]:
                    flatten.append(x)
            if key == 'P':
                dictionaries[key] = list(range(min(flatten), max(flatten)+1))
            else:
                dictionaries[key] = sorted(list(set(flatten)))

        self.dataset = dataset
        self.dictionaries = dictionaries

        print(self.dictionaries)

    def saveData(self):
        pickle.dump([self.dataset, self.dictionaries], open(self.datapath+self.corpus+"/data.pkl", "wb" ))

    def loadData(self):
        if os.path.exists(self.datapath+self.corpus+"/data.pkl"):
            print("Loading Data")
            self.dataset, self.dictionaries = pickle.load( open(self.datapath+self.corpus+"/data.pkl", "rb" ) )
        else:
            print("No preprocessed data for %s has been found -> Processing %s"%(self.corpus, self.datapath+self.corpus))
            self.processData()
            self.saveData()
        #Add start/end tags
        tag='START/END'
        if tag not in self.dictionaries['dT']:
            self.dictionaries['dT'].append(tag)
            self.dictionaries['T'].append(tag)
            self.dictionaries['P'].append(tag)

    def computeIntervalRepr(self):
        
        for label in self.dataset.keys():
            dPseq = []
            p_nm1 = self.dataset[label]['P'][0]
            for n, p_n in enumerate(self.dataset[label]['P']):
                dPseq.append(p_n-p_nm1)
                p_nm1 = p_n
            self.dataset[label]['dP'] = dPseq

    def checkRepresentation(self):
        label = str(np.random.choice(list(self.dataset.keys()), 1)[0])
        utils.writeMIDI(self.dataset[label]['dT'][1:-1], self.dataset[label]['T'][1:-1], self.dataset[label]['P'][1:-1], path="../data/"+self.corpus+"/", label=label, tag='retrieved')



    def ANN2data(self, XdTs, XTs, XPs):
        """
        Translate back from one-hot matrices to note sequences
        """
        dTs = []
        Ts = []
        Ps = []

        tag='START/END'
        if tag not in self.dictionaries['dT']:
            self.dictionaries['dT'].append(tag)
            self.dictionaries['T'].append(tag)
            self.dictionaries['P'].append(tag)

        for XdT, XT, XP in zip(XdTs[:,1:], XTs[:,1:], XPs[:,1:]):
            xdT = np.where(XdT == 1)[1]
            xT = np.where(XT == 1)[1]
            xP = np.where(XP == 1)[1]
            dT = []
            T = []
            P = []
            for dt, t, p in zip(xdT, xT, xP):
                dt = self.dictionaries['dT'][dt]
                t = self.dictionaries['T'][t]
                p = self.dictionaries['P'][p]

                dT.append(dt)
                T.append(t)
                P.append(p)

            dTs.append(dT)
            Ts.append(T)
            Ps.append(P)
            
        return dTs, Ts, Ps

    def getMaxLen(self):
        seqlens = [len(score['P']) for score in self.dataset.values()]
        len95Perc = int(np.mean(seqlens)+2*np.std(seqlens))
        maxlen = len95Perc + int(self.TBPTT_size-len95Perc%(self.TBPTT_size))
        self.maxlen = maxlen
        return maxlen

    def data2ANN(self):

        if self.dataset is None:
            self.loadData()


        #Add start/end tags
        tag='START/END'
        if tag not in self.dictionaries['dT']:
            self.dictionaries['dT'].append(tag)
            self.dictionaries['T'].append(tag)
            self.dictionaries['P'].append(tag)
        for label, score in self.dataset.items():
            if tag not in score['dT']:
                score['dT'].insert(0,tag)
                score['T'].insert(0,tag)
                score['P'].insert(0,tag)
                score['dT'].append(tag)
                score['T'].append(tag)
                score['P'].append(tag)


        xdT, xT, xP, labels = tokenize(self.dataset, self.dictionaries)

        P = [np_utils.to_categorical(x, len(self.dictionaries['P'])) for x in xP]
        T = [np_utils.to_categorical(x, len(self.dictionaries['T'])) for x in xT]
        dT = [np_utils.to_categorical(x, len(self.dictionaries['dT'])) for x in xdT]

        seqlens = [len(X) for X in P]
        len95Perc = int(np.mean(seqlens)+2*np.std(seqlens))

        maxlen = len95Perc + int(self.TBPTT_size-len95Perc%(self.TBPTT_size)) + 1

        dT = pad_sequences(dT, value=0., dtype="int32", padding="post", truncating="post", maxlen=maxlen)
        T = pad_sequences(T, value=0., dtype="int32", padding="post", truncating="post", maxlen=maxlen)
        P = pad_sequences(P, value=0., dtype="int32", padding="post", truncating="post", maxlen=maxlen)
        #print("Longest melody length [note]: %i, mean: %i, std: %i, selected length: %i"%(max(seqlens),np.mean(seqlens),np.std(seqlens), maxlen))

        XdT = np.asarray([x[:-1] for x in dT], dtype=int)
        YdT = np.asarray([x[1:] for x in dT], dtype=int)

        XT = np.asarray([x[:-1] for x in T], dtype=int)
        YT = np.asarray([x[1:] for x in T], dtype=int)

        XP = np.asarray([x[:-1] for x in P], dtype=int)
        YP = np.asarray([x[1:] for x in P], dtype=int)

        print("\tFinal I/O data shape:")
        print('\tdT', XdT.shape)
        print('\tT', XT.shape)
        print('\tP', XP.shape)


        self.TBPTT_steps = []
        for X in XP:
            steps = 0
            while np.sum(X[steps*self.TBPTT_size:(steps+1)*self.TBPTT_size]) > 0:
                steps += 1
            self.TBPTT_steps.append(steps)
        self.TBPTT_steps = np.asarray(self.TBPTT_steps)

        IO = {'XP': XP, 'YP': YP, 'XT': XT, 'YT': YT, 'XdT': XdT, 'YdT': YdT, 'TBPTT_steps': self.TBPTT_steps, "labels": labels}

        self.IO = IO
        self.maxlen = maxlen - 1


    def buildModel(self, dropout=0., recurrent_dropout=0.):
        print('\nBuilding Model')

        X = dict()
        M = dict()
        H = dict()
        Y = dict()
        
        dTvocsize = len(self.dictionaries['dT'])
        Tvocsize = len(self.dictionaries['T'])
        Pvocsize = len(self.dictionaries['P'])
        
        X['dT_n'] = Input(batch_shape=(self.batch_size, self.TBPTT_size, dTvocsize), name='XdT_n')
        X['dT_np1'] = Input(batch_shape=(self.batch_size, self.TBPTT_size, dTvocsize), name='XdT_np1')
        X['T_n'] = Input(batch_shape=(self.batch_size, self.TBPTT_size, Tvocsize), name='XT_n') 
        X['T_np1'] = Input(batch_shape=(self.batch_size, self.TBPTT_size, Tvocsize), name='XT_np1') 
        X['P_n'] = Input(batch_shape=(self.batch_size, self.TBPTT_size, Pvocsize), name='XP_n')

        M['dT_n'] = Masking(mask_value=0.)(X['dT_n'])
        M['dT_np1'] = Masking(mask_value=0.)(X['dT_np1'])
        M['T_n'] = Masking(mask_value=0.)(X['T_n'])
        M['T_np1'] = Masking(mask_value=0.)(X['T_np1'])
        M['P_n'] = Masking(mask_value=0.)(X['P_n'])

        X['H1'] = Concatenate()([M['dT_n'], M['T_n'], M['P_n']])
        H['1'] =  GRU(128, 
            return_sequences=True, 
            stateful=True
            )(X['H1'])

        X['H2'] = Concatenate()([M['dT_n'], M['T_n'], M['P_n'], H['1']])
        H['2'] =  GRU(128, 
            return_sequences=True, 
            stateful=True
            )(X['H2'])

        X['H3'] = Concatenate()([M['dT_n'], M['T_n'], M['P_n'], H['2']])
        H['3'] =  GRU(128, 
            return_sequences=True, 
            stateful=True
            )(X['H3'])

        X['H4'] = H['3']
        H['4'] =  GRU(128, 
            return_sequences=True, 
            stateful=True
            )(X['H4'])
   
        X['HP'] = Concatenate()([H['1'], H['2'], H['3'], H['4'], M['dT_np1'], M['T_np1']])
        H['P'] = TimeDistributed(Dense(Pvocsize, activation='relu'))(X['HP'])
        Y['P'] = TimeDistributed(Dense(Pvocsize, activation='softmax'), name='YP')(H['P'])

        self.Pmodel = Model(inputs = [X['dT_n'], X['T_n'], X['P_n'], X['dT_np1'], X['T_np1']], outputs = [Y['P']])
        self.Pmodel.compile(
            loss='categorical_crossentropy', 
            optimizer=Adam(),
            metrics=['acc'],
            loss_weights=[.6])
    
        # H['1'].trainable = False
        # H['2'].trainable = False
        # H['3'].trainable = False

        X['HdT'] = H['1']
        H['dT'] = TimeDistributed(Dense(dTvocsize, activation='relu'))(X['HdT'])
        Y['dT'] = TimeDistributed(Dense(dTvocsize, activation='softmax'), name='YdT')(H['dT'])
        self.dTmodel = Model(inputs = [X['dT_n'], X['T_n'], X['P_n']], outputs = [Y['dT']])
        self.dTmodel.compile(
            loss='categorical_crossentropy', 
            optimizer=Adam(),
            metrics=['acc'],
            loss_weights=[.1])

        X['HT'] = Concatenate()([H['1'], H['2'], M['dT_np1']])
        H['T'] = TimeDistributed(Dense(Tvocsize, activation='relu'))(X['HT'])
        Y['T'] = TimeDistributed(Dense(Tvocsize, activation='softmax'), name='YT')(H['T'])
        self.Tmodel = Model(inputs = [X['dT_n'], X['T_n'], X['P_n'], X['dT_np1']], outputs = [Y['T']])
        self.Tmodel.compile(
            loss='categorical_crossentropy', 
            optimizer=Adam(),
            metrics=['acc'],
            loss_weights=[.3])


        

        #H['2'].trainable = False
        #H['T'].trainable = False

        
    def loadModel(self, tag=None):
        print('\nLoading Model')
        if tag is None:
            self.dTmodel = load_model(self.loadmodelpath+self.corpus+"/dTmodel.bp")
            self.Tmodel = load_model(self.loadmodelpath+self.corpus+"/Tmodel.bp")
            self.Pmodel = load_model(self.loadmodelpath+self.corpus+"/Pmodel.bp")
        else:
            self.dTmodel = load_model(self.loadmodelpath+self.corpus+"/"+tag+"_dTmodel.bp")
            self.Tmodel = load_model(self.loadmodelpath+self.corpus+"/"+tag+"_Tmodel.bp")
            self.Pmodel = load_model(self.loadmodelpath+self.corpus+"/"+tag+"_Pmodel.bp")

        self.log = pickle.load(open(self.loadmodelpath+self.corpus+"/log.bp", "rb" ))
        self.best_val_accP = max(self.log['val_P_acc'])
        self.best_tr_accP = max(self.log['P_acc'])
        self.best_val_accT = max(self.log['val_T_acc'])
        self.best_tr_accT = max(self.log['T_acc'])
        self.best_val_accdT = max(self.log['val_dT_acc'])
        self.best_tr_accdT = max(self.log['dT_acc'])
        self.split = pickle.load(open(self.loadmodelpath+self.corpus+"/split.bp", "rb"))
        self.TBPTT_steps = [int(x) for x in self.split['train'].keys()]

    def saveModel(self, tag=None):
        if tag is not None:
            self.dTmodel.save(self.outpath+tag+"_dTmodel.bp")
            self.Tmodel.save(self.outpath+tag+"_Tmodel.bp")
            self.Pmodel.save(self.outpath+tag+"_Pmodel.bp")
        else:
            self.dTmodel.save(self.outpath+"dTmodel.bp")
            self.Tmodel.save(self.outpath+"Tmodel.bp")
            self.Pmodel.save(self.outpath+"Pmodel.bp")
        pickle.dump(self.log, open(self.outpath+"log.bp", "wb" ))
        pickle.dump(self.split, open(self.outpath+"split.bp", "wb" ))

    def transpose(self):
        for i,XP in enumerate(self.IO['XP']):
            collapsed = np.sum(XP, axis=0)
            active_pitches = np.where(collapsed>0)[0]
            upperbound = active_pitches[-1]
            lowerbound = active_pitches[0]
            possible_shifts = range(0-lowerbound, len(self.dictionaries['P'])-upperbound)
            if len(possible_shifts) == 0:
                possible_shifts.append(0)
            shift = np.random.choice(possible_shifts)
            self.IO['XP'][i] = np.roll(XP, shift, axis=1)
            self.IO['YP'][i] = np.roll(self.IO['YP'][i], shift, axis=1)


    def trainModel(self, epochs=2000, validation_split=0.1):

        if self.IO is None:
            self.data2ANN()

        dTvocsize = len(self.dictionaries['dT'])
        Tvocsize = len(self.dictionaries['T'])
        Pvocsize = len(self.dictionaries['P'])

        print('\nTraining Model')

        nb_samples = self.IO['XP'].shape[0]
        all_idxes = np.asarray(range(nb_samples))
        all_batch_idxes = {'train': {}, 'valid': {}}

        if self.split is None:
            idxes_to_add = []
            for step in set(self.TBPTT_steps):
                #print(step)
                idxes = all_idxes[np.where(self.TBPTT_steps==step)]
                np.random.shuffle(idxes)
                idxes = list(idxes)
                if idxes_to_add:
                    idxes.extend(idxes_to_add)
                    idxes_to_add = []
                #print(idxes)
                nb_samples = len(idxes)
                if nb_samples < 10:#not enough samples for this length
                    idxes_to_add = idxes
                    continue
                #print(nb_samples)
                split_idx = int((1 - validation_split) * nb_samples)
                #split_idx -= split_idx % self.batch_size
                all_batch_idxes['train'][str(step)] = idxes[:split_idx]
                #print(all_batch_idxes['train'][str(step)])
                all_batch_idxes['train'][str(step)].extend(
                                            np.random.choice(idxes[:split_idx], 
                                                    self.batch_size - len(all_batch_idxes['train'][str(step)]) % self.batch_size))
                #print(all_batch_idxes['train'][str(step)])
                all_batch_idxes['valid'][str(step)] = idxes[split_idx:]
                all_batch_idxes['valid'][str(step)].extend(
                                            np.random.choice(idxes[split_idx:], 
                                                    self.batch_size - len(all_batch_idxes['valid'][str(step)]) % self.batch_size))

            self.split = all_batch_idxes
        else:
            all_batch_idxes = self.split
            
        
        self.TBPTT_steps = [int(x) for x in self.split['train'].keys()]
        
        
        for epoch in range(epochs):
            tr_epoch_res = []
            self.transpose()
            np.random.shuffle(self.TBPTT_steps)
            for step_number in self.TBPTT_steps:
                #Reshape for batches
                np.random.shuffle(all_batch_idxes['train'][str(step_number)])
                batch_idxes = np.reshape(all_batch_idxes['train'][str(step_number)], (-1,self.batch_size))
                batch_nbr_per_epoch = len(batch_idxes)
                for batch, idxes in enumerate(batch_idxes):

                    # self.Pmodel.reset_states()
                    # for step in range(step_number):
                    #     lossP, accP = self.Pmodel.train_on_batch({'XdT_n': self.IO['XdT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size], 
                    #                                 'XT_n': self.IO['XT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size], 
                    #                                 'XP_n': self.IO['XP'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size],
                    #                                 'XdT_np1': self.IO['YdT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size],
                    #                                 'XT_np1': self.IO['YT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size]}, 
                    #                                {'YP': self.IO['YP'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size]})


                    self.dTmodel.reset_states()
                    batch_lossdT = []
                    batch_accdT = []
                    for step in range(step_number):
                        lossdT, accdT = self.dTmodel.train_on_batch({'XdT_n': self.IO['XdT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size], 
                                                    'XT_n': self.IO['XT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size], 
                                                    'XP_n': self.IO['XP'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size]}, 
                                                   {'YdT': self.IO['YdT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size]})
                        batch_lossdT.append(lossdT)
                        batch_accdT.append(accdT)

                    self.Tmodel.reset_states()
                    batch_lossT = []
                    batch_accT = []
                    for step in range(step_number):
                        lossT, accT = self.Tmodel.train_on_batch({'XdT_n': self.IO['XdT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size], 
                                                    'XT_n': self.IO['XT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size], 
                                                    'XP_n': self.IO['XP'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size],
                                                    'XdT_np1': self.IO['YdT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size]}, 
                                                   {'YT': self.IO['YT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size]})
                        batch_lossT.append(lossT)
                        batch_accT.append(accT)

                    self.Pmodel.reset_states()
                    batch_lossP = []
                    batch_accP = []
                    for step in range(step_number):
                        lossP, accP = self.Pmodel.train_on_batch({'XdT_n': self.IO['XdT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size], 
                                                    'XT_n': self.IO['XT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size], 
                                                    'XP_n': self.IO['XP'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size],
                                                    'XdT_np1': self.IO['YdT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size],
                                                    'XT_np1': self.IO['YT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size]}, 
                                                   {'YP': self.IO['YP'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size]})
                        batch_lossP.append(lossP)
                        batch_accP.append(accP)

                    lossdT = np.mean(batch_lossdT) 
                    lossT = np.mean(batch_lossT) 
                    lossP = np.mean(batch_lossP) 

                    accdT = np.mean(batch_accdT)
                    accT = np.mean(batch_accT)
                    accP = np.mean(batch_accP)
                    

                    res = [.1*lossdT+.3*lossT+.6*lossP, lossdT, lossT, lossP, accdT, accT, accP] 
                    tr_epoch_res.append(res)

            tr_epoch_res = np.nanmean(tr_epoch_res, axis=0)
            tr_loss, tr_lossdT, tr_lossT, tr_lossP, tr_accdT, tr_accT, tr_accP = tr_epoch_res
            print("Epoch %i/%i\t loss: %.2f - dT: %.2f - T: %.2f - P: %.2f" %
                 (epoch+1,epochs,tr_loss,tr_accdT, tr_accT, tr_accP))
            
            val_epoch_res = []
            for step_number in self.TBPTT_steps:
                #Reshape for batches
                np.random.shuffle(all_batch_idxes['valid'][str(step_number)])
                batch_idxes = np.reshape(all_batch_idxes['valid'][str(step_number)], (-1,self.batch_size))
                batch_nbr_per_epoch = len(batch_idxes)
                for batch, idxes in enumerate(batch_idxes):
                    self.dTmodel.reset_states()
                    batch_lossdT = []
                    batch_accdT = []
                    for step in range(step_number):
                        lossdT, accdT = self.dTmodel.test_on_batch({'XdT_n': self.IO['XdT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size], 
                                                    'XT_n': self.IO['XT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size], 
                                                    'XP_n': self.IO['XP'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size]}, 
                                                   {'YdT': self.IO['YdT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size]})
                        batch_lossdT.append(lossdT)
                        batch_accdT.append(accdT)

                    self.Tmodel.reset_states()
                    batch_lossT = []
                    batch_accT = []
                    for step in range(step_number):   
                        lossT, accT = self.Tmodel.test_on_batch({'XdT_n': self.IO['XdT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size], 
                                                    'XT_n': self.IO['XT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size], 
                                                    'XP_n': self.IO['XP'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size],
                                                    'XdT_np1': self.IO['YdT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size]}, 
                                                   {'YT': self.IO['YT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size]})
                        batch_lossT.append(lossT)
                        batch_accT.append(accT)
                    self.Pmodel.reset_states()
                    batch_lossP = []
                    batch_accP = []
                    for step in range(step_number):
                        lossP, accP = self.Pmodel.test_on_batch({'XdT_n': self.IO['XdT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size], 
                                                    'XT_n': self.IO['XT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size], 
                                                    'XP_n': self.IO['XP'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size],
                                                    'XdT_np1': self.IO['YdT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size],
                                                    'XT_np1': self.IO['YT'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size]}, 
                                                   {'YP': self.IO['YP'][idxes,step*self.TBPTT_size:(step+1)*self.TBPTT_size]})
                        batch_lossP.append(lossP)
                        batch_accP.append(accP)
                    lossdT = np.nanmean(batch_lossdT) 
                    lossT = np.nanmean(batch_lossT) 
                    lossP = np.nanmean(batch_lossP) 

                    accdT = np.nanmean(batch_accdT)
                    accT = np.nanmean(batch_accT)
                    accP = np.nanmean(batch_accP)
                    

                    res = [.1*lossdT+.3*lossT+.6*lossP, lossdT, lossT, lossP, accdT, accT, accP] 
                    val_epoch_res.append(res)
            val_epoch_res = np.nanmean(val_epoch_res, axis=0)
            val_loss, val_lossdT, val_lossT, val_lossP, val_accdT, val_accT, val_accP = val_epoch_res
            print("\t\tloss: %.2f - dT: %.2f - T: %.2f - P: %.2f" %
                 (val_loss,val_accdT, val_accT, val_accP))
            
            self.log['loss'].append(tr_loss)
            self.log['dT_acc'].append(tr_accdT)
            self.log['T_acc'].append(tr_accT)
            self.log['P_acc'].append(tr_accP)
            self.log['dT_loss'].append(tr_lossdT)
            self.log['T_loss'].append(tr_lossT)
            self.log['P_loss'].append(tr_lossP)
            self.log['val_loss'].append(val_loss)
            self.log['val_dT_acc'].append(val_accdT)
            self.log['val_T_acc'].append(val_accT)
            self.log['val_P_acc'].append(val_accP)
            self.log['val_dT_loss'].append(val_lossdT)
            self.log['val_T_loss'].append(val_lossT)
            self.log['val_P_loss'].append(val_lossP)
            
            if  val_accP > self.best_val_accP :
                self.best_val_accP = val_accP
                self.Pmodel.save(self.outpath+"Pmodel.bp")
                pickle.dump(self.log, open(self.outpath+"log.bp", "wb" ))
                pickle.dump(self.split, open(self.outpath+"split.bp", "wb" ))

            if  tr_accP > self.best_tr_accP:
                self.best_tr_accP = tr_accP
                self.Pmodel.save(self.outpath+"train_Pmodel.bp")
                pickle.dump(self.log, open(self.outpath+"log.bp", "wb" ))
                pickle.dump(self.split, open(self.outpath+"split.bp", "wb" ))

            if  val_accT > self.best_val_accT :
                self.best_val_accT = val_accT
                self.Tmodel.save(self.outpath+"Tmodel.bp")
                pickle.dump(self.log, open(self.outpath+"log.bp", "wb" ))
                pickle.dump(self.split, open(self.outpath+"split.bp", "wb" ))

            if  tr_accT > self.best_tr_accT:
                self.best_tr_accT = tr_accT
                self.Tmodel.save(self.outpath+"train_Tmodel.bp")
                pickle.dump(self.log, open(self.outpath+"log.bp", "wb" ))
                pickle.dump(self.split, open(self.outpath+"split.bp", "wb" ))

            if  val_accdT > self.best_val_accdT :
                self.best_val_accdT = val_accdT
                self.dTmodel.save(self.outpath+"dTmodel.bp")
                pickle.dump(self.log, open(self.outpath+"log.bp", "wb" ))
                pickle.dump(self.split, open(self.outpath+"split.bp", "wb" ))

            if  tr_accdT > self.best_tr_accdT:
                self.best_tr_accdT = tr_accdT
                self.dTmodel.save(self.outpath+"train_dTmodel.bp")
                pickle.dump(self.log, open(self.outpath+"log.bp", "wb" ))
                pickle.dump(self.split, open(self.outpath+"split.bp", "wb" ))

        
    def generate(self, note_len=1000, until_all_ended=True, temperature=0.5, seed=None):
        if until_all_ended:
            note_len = self.IO['XP'].shape[1]

        tag='START/END'
        if tag not in self.dictionaries['dT']:
            self.dictionaries['dT'].append(tag)
            self.dictionaries['T'].append(tag)
            self.dictionaries['P'].append(tag)

        dTvocsize = len(self.dictionaries['dT'])
        Tvocsize = len(self.dictionaries['T'])
        Pvocsize = len(self.dictionaries['P'])
        n_ex = self.batch_size

        ended = [False]*n_ex

        XdTs_hat = np.zeros((n_ex, note_len+1, dTvocsize), dtype = int)
        XdTs_hat[:,0,-1] = 1
        XdTs_probs = np.zeros((n_ex, note_len+1, dTvocsize), dtype = float)

        XTs_hat = np.zeros((n_ex, note_len+1, Tvocsize), dtype = int)
        XTs_hat[:,0,-1] = 1
        XTs_probs = np.zeros((n_ex, note_len+1, Tvocsize), dtype = float)

        XPs_hat = np.zeros((n_ex, note_len+1, Pvocsize), dtype = int)
        XPs_hat[:,0,-1] = 1
        XPs_probs = np.zeros((n_ex, note_len+1, Pvocsize), dtype = float)

        if seed is not None:
            seed_len = len(seed['dT'])
            for i in range(n_ex):
                XdTs_hat[i,1:seed_len+1] = seed['dT']
                XTs_hat[i,1:seed_len+1] = seed['T']
                XPs_hat[i,1:seed_len+1] = seed['P']

        xdt_t = np.zeros((n_ex, self.TBPTT_size, dTvocsize), dtype=int)
        xt_t = np.zeros((n_ex, self.TBPTT_size, Tvocsize), dtype=int)
        xp_t = np.zeros((n_ex, self.TBPTT_size, Pvocsize), dtype=int)
        xdt_tp1 = np.zeros((n_ex, self.TBPTT_size, dTvocsize), dtype=int)
        xt_tp1 = np.zeros((n_ex, self.TBPTT_size, Tvocsize), dtype=int)

        self.dTmodel.reset_states()
        self.Tmodel.reset_states()
        self.Pmodel.reset_states()

        try:
            last_note = 0
            for t in tqdm(range(note_len)):  
                xdt_t[:,0] = XdTs_hat[:,t]
                xt_t[:,0] = XTs_hat[:,t]
                xp_t[:,0] = XPs_hat[:,t]

                probs = self.dTmodel.predict([xdt_t, xt_t, xp_t])
                XdTs_probs[:,t] = probs[:, 0]  
                for idx in range(n_ex):  
                    dT_np1 = sample(probs[idx, 0], temperature=temperature)
                    if np.sum(XdTs_hat[idx, t+1]) == 0:
                        XdTs_hat[idx, t+1, dT_np1] = 1
                        if self.dictionaries['dT'][dT_np1] == 'START/END':
                            if ended[idx] == False:
                                print("%i dT ended @note #%i"%(idx+1, t))
                            ended[idx] = True 

                xdt_tp1[:,0] = XdTs_hat[:,t+1]

                probs = self.Tmodel.predict([xdt_t, xt_t, xp_t, xdt_tp1])
                XTs_probs[:,t] = probs[:, 0]          
                for idx in range(n_ex):  
                    T_np1 = sample(probs[idx, 0], temperature=temperature)
                    if np.sum(XTs_hat[idx, t+1]) == 0:
                        XTs_hat[idx, t+1, T_np1] = 1
                        if self.dictionaries['T'][T_np1] == 'START/END':
                            if ended[idx] == False:
                                print("%i T ended @note #%i"%(idx+1, t))
                            ended[idx] = True 
                xt_tp1[:,0] = XTs_hat[:,t+1]

                probs = self.Pmodel.predict([xdt_t, xt_t, xp_t, xdt_tp1, xt_tp1])
                for idx in range(n_ex):  
                    if not ended[idx]:
                        XPs_probs[idx,t] = probs[idx, 0]
                    if np.sum(XPs_hat[idx, t+1]) == 0:
                        P_np1 = sample(probs[idx, 0], temperature=temperature)
                        XPs_hat[idx, t+1, P_np1] = 1.
                        if self.dictionaries['P'][P_np1] == 'START/END':
                            if ended[idx] == False:
                                print("%i P ended @note #%i"%(idx+1, t))
                            ended[idx] = True 



                last_note = t
                if until_all_ended == True and np.sum(ended) == n_ex:
                    break
        except KeyboardInterrupt:
            print("ctrl-c ended @note %i"%(t))
            pass
        print("End generating: %i/%i song ended"%(np.sum(ended),n_ex))
        return [XdTs_hat[:,:last_note], XTs_hat[:,:last_note], XPs_hat[:,:last_note], XdTs_probs[:,:last_note], XTs_probs[:,:last_note], XPs_probs[:,:last_note]]

    
    def deleteGen(self): 
        print(self.genpath)
        if os.path.exists(self.genpath):
            shutil.rmtree(self.genpath)
        if not os.path.exists(self.genpath):
            os.makedirs(self.genpath)

In [4]:
import shutil
# m.deleteGen()


In [5]:
import midi
import copy
import numpy as np
import mido

In [6]:

def mergeTrack(s):
    """
    Merge all tracks in s in a single one.
    """
    singletrack = midi.Track()
    events = []
    for i, track in enumerate(s):
        t = 0
        for event in track:
            t += event.tick
            if event.name in ['Note On', 'Note Off']:
                candidate = {'t': t, 'event': event}
                if candidate not in events:
                    events.append(candidate)


    events = sorted(events, key=lambda k: (k['t'], k['event'].data[0]))
    tick = 0
    for e in events:
        e['event'].tick = e['t'] - tick
        tick = e['t']
        singletrack.append(e['event'])
    return singletrack


In [7]:
def tokenize(dataset, dictionaries):
    xP = []
    xT = []
    xdT = []
    labels = []
    for label, melody in dataset.items():
        xP.append([dictionaries["P"].index(x) for x in melody["P"]])
        xT.append([dictionaries["T"].index(x) for x in melody["T"]])
        xdT.append([dictionaries["dT"].index(x) for x in melody["dT"]])
        labels.append(label)
    return xdT, xT, xP, labels

In [8]:
def sample(preds, temperature=1.):
    if temperature == 0.:
        return np.argmax(preds)
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)
    out = np.zeros(shape=preds.shape, dtype='float32')
    out[song_idx,t,np.argmax(probas)] = 1.
    return out


In [9]:

def longMIDI(dtseqs, Tseqs, pitchseqs, path, label="1", tag="all", resolution=1024, write_all=False, bpm=80):
    ended = False
    # Instantiate a MIDI Pattern (contains a list of tracks)
    pattern = midi.Pattern(format = 0, resolution = resolution)
    # Instantiate a MIDI Track (contains a list of MIDI events)
    track = midi.Track()
    # Append the track to the pattern
    pattern.append(track)
    tick = 0
    Events = []

    #Set the tempo
    e = midi.SetTempoEvent()
    e.set_bpm(bpm)
    track.append(e)

    i = 0
    number = 0
    for dtseq, Tseq, pitchseq in zip(dtseqs, Tseqs, pitchseqs):
        i+=1
        last_dur = 0
        song_ended = False
        for dt, T, p in zip(dtseq, Tseq, pitchseq):
            if dt == 'START/END' or T == 'START/END' or p == 'START/END':
                song_ended = True
                tick = tick + int(resolution*last_dur) + resolution*4
                break
            tick = tick + int(dt*resolution)
            Events.append({'t': tick, 'p': p, 'm': 'ON'})
            Events.append({'t': tick+int(T*resolution), 'p': p, 'm': 'OFF'})
            last_dur = T
        tick = tick + int(resolution*last_dur) + resolution*4
        if song_ended:
            writeMIDI(dtseq, Tseq, pitchseq, path=path, label=label, tag=str(i), resolution=resolution, bpm=bpm)
            number += 1
        else:
            writeMIDI(dtseq, Tseq, pitchseq, path=path, label=label, tag=str(i)+"_NoEnd", resolution=resolution, bpm=bpm)
    Events = sorted(Events, key=lambda k: k['t'])
    tick = 0
    onehour = False
    for event in Events:
        if event['m'] == 'ON':
            e =  midi.NoteOnEvent(tick=event['t']-tick, velocity=90, pitch=event['p'])
        if event['m'] == 'OFF':
            e =  midi.NoteOffEvent(tick=event['t']-tick, velocity=90, pitch=event['p'])
        track.append(e)
        tick = event['t']
        if tick * (60. / (120.*resolution)) > 3600.:
            onehour = True
            break
      
    # Add the end of track event, append it to the track
    eot = midi.EndOfTrackEvent(tick=1)
    track.append(eot)
    # Save the pattern to disk
    if write_all:
        midi.write_midifile(path+label+"_"+tag+".mid", pattern)
        midifile = mido.MidiFile(path+label+"_"+tag+".mid")
    
        print(midifile.length, tick * (60. / (120.*resolution)), len(midifile.tracks), midifile.type)
        if onehour:
            print("Long enough...YEY")
    return number


In [10]:

def writeMIDI(dtseq, Tseq, pitchseq, path, label="1", tag="1", resolution=256, bpm=100):

    # Instantiate a MIDI Pattern (contains a list of tracks)
    pattern = midi.Pattern(format = 0, resolution = resolution)
    # Instantiate a MIDI Track (contains a list of MIDI events)
    track = midi.Track()
    # Append the track to the pattern
    pattern.append(track)
    tick = 0
    Events = []

    #Set the tempo
    e = midi.SetTempoEvent()
    e.set_bpm(bpm)
    track.append(e)
    
    #Timing
    timeperbeat = 60. / bpm#[s]
    timepertick = timeperbeat/resolution
    longenough = False
    for dt, T, p in zip(dtseq, Tseq, pitchseq):
        if tick >= resolution*8:#more than 8 beats
            longenough = True
        if dt == 'START/END' or T == 'START/END' or p == 'START/END':
            break
        tick = tick + int(dt*resolution)
        Events.append({'t': tick, 'p': p, 'm': 'ON'})
        Events.append({'t': tick+int(T*resolution), 'p': p, 'm': 'OFF'})

    Events = sorted(Events, key=lambda k: k['t'])
    tick = 0
    for event in Events:
        if event['m'] == 'ON':
            e =  midi.NoteOnEvent(tick=event['t']-tick, velocity=90, pitch=event['p'])
        if event['m'] == 'OFF':
            e =  midi.NoteOffEvent(tick=event['t']-tick, velocity=90, pitch=event['p'])
        track.append(e)
        tick = event['t']
        

    # Add the end of track event, append it to the track
    eot = midi.EndOfTrackEvent(tick=1)
    track.append(eot)
    if longenough:
        # Save the pattern to disk
        print("longenough")
        # print(path)
        midi.write_midifile(path+label+"_"+tag+".mid", pattern)
    return pattern


In [11]:

def parseMIDI(midi_file_path):
    """
    Process the MIDI in midi_file_path to extract
    the sequence of notes (dT, T, P). Timing is in 
    MIDI tick representation. The function also 
    returns the tick per beat (tpb) metaparameter
    """ 
    s = midi.read_midifile(midi_file_path)
    tpb = float(s.resolution)
    events = mergeTrack(s)
    T = []
    P = []
    dT = []
    dt = 0
    for n, event in enumerate(events):
        if event.name == 'Note On' and event.data[1] > 0:
            pitch_n = event.data[0]
            n2 = n
            duration_n = 0
            while True:
                n2 += 1
                if n2 > (len(events)-1):
                    break
                duration_n += events[n2].tick
                if events[n2].data[0] == pitch_n and events[n2].name == 'Note Off':
                    break
                if events[n2].data[0] == pitch_n and events[n2].name == 'Note On' and events[n2].data[1] == 0:
                    break
            if duration_n > 0.:
                P.append(pitch_n)
                T.append(duration_n)
                dT.append(event.tick+dt)
            dt = 0
        elif event.name == 'Note Off' or event.data[1] == 0:
            dt += event.tick
    
    #Tick (integer) to beat fraction (float)
    dT = [float(dt)/tpb for dt in dT]
    T = [float(t)/tpb for t in T]
    return dT, T, P, tpb


In [31]:

def findClosest(val, vec):
    """
    find the element closest to val in vec 
    """
    diff = [abs(float(el)-float(val)) for el in vec]
    idx = np.argmin(diff)
    return vec[idx]


In [40]:
m = DeepChopin("Chopin")

loading data:

In [41]:
m.loadData()

Loading Data


In [42]:
m.data2ANN()

	Final I/O data shape:
	dT (20, 3072, 151)
	T (20, 3072, 357)
	P (20, 3072, 79)


if generate music with pretrained model:

In [43]:
m.loadModel() 


Loading Model


In [44]:
# pip show h5py

In [45]:
XdTs_hat, XTs_hat, XPs_hat, XdTs_probs, XTs_probs, XPs_probs = m.generate()

  0%|          | 1/3072 [00:03<3:08:15,  3.68s/it]/home/vmarina/.conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
 17%|█▋        | 508/3072 [02:02<10:02,  4.26it/s]

18 dT ended @note #507


100%|██████████| 3072/3072 [12:01<00:00,  4.29it/s]

End generating: 1/32 song ended


In [46]:
dTs, Ts, Ps = m.ANN2data(XdTs_hat, XTs_hat, XPs_hat)

In [47]:
longMIDI(dTs, Ts, Ps, path=m.genpath, label='generated', bpm=90)

longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough
longenough


1

here goes the training part

In [28]:
m.buildModel()


Building Model



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [29]:
m.trainModel()


Training Model



2022-05-24 17:48:20.681236: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2022-05-24 17:48:20.685773: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2998330000 Hz
2022-05-24 17:48:20.686611: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x56231ee99ad0 executing computations on platform Host. Devices:
2022-05-24 17:48:20.686634: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2022-05-24 17:48:20.882562: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not 

Epoch 1/2000	 loss: 1.91 - dT: 0.28 - T: 0.13 - P: 0.04
		loss: 1.92 - dT: 0.35 - T: 0.04 - P: 0.02
Epoch 2/2000	 loss: 1.80 - dT: 0.32 - T: 0.12 - P: 0.04
		loss: 1.71 - dT: 0.35 - T: 0.36 - P: 0.07
Epoch 3/2000	 loss: 1.76 - dT: 0.33 - T: 0.17 - P: 0.04
		loss: 1.79 - dT: 0.35 - T: 0.04 - P: 0.02
Epoch 4/2000	 loss: 1.74 - dT: 0.35 - T: 0.16 - P: 0.04
		loss: 1.70 - dT: 0.36 - T: 0.36 - P: 0.07
Epoch 5/2000	 loss: 1.71 - dT: 0.36 - T: 0.19 - P: 0.06
		loss: 1.68 - dT: 0.37 - T: 0.31 - P: 0.07
Epoch 6/2000	 loss: 1.68 - dT: 0.37 - T: 0.19 - P: 0.07
		loss: 1.62 - dT: 0.35 - T: 0.35 - P: 0.06
Epoch 7/2000	 loss: 1.64 - dT: 0.38 - T: 0.20 - P: 0.08
		loss: 1.62 - dT: 0.44 - T: 0.38 - P: 0.06
Epoch 8/2000	 loss: 1.60 - dT: 0.39 - T: 0.20 - P: 0.09
		loss: 1.57 - dT: 0.45 - T: 0.36 - P: 0.06
Epoch 9/2000	 loss: 1.56 - dT: 0.40 - T: 0.22 - P: 0.11
		loss: 1.60 - dT: 0.49 - T: 0.37 - P: 0.07
Epoch 10/2000	 loss: 1.54 - dT: 0.41 - T: 0.23 - P: 0.11
		loss: 1.57 - dT: 0.44 - T: 0.38 - P: 0.09